# Hand Gesture Recognition - Minimal Version
## Training model for 5 gesture classes

In [ ]:
# Import necessary libraries
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import datetime
import imageio
import pickle

# Deep Learning libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense, Dropout, GRU, GlobalAveragePooling2D, 
    TimeDistributed
)
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2

# Suppress TensorFlow warnings (optional)
tf.get_logger().setLevel('ERROR')

print("Libraries imported successfully")
print(f"TensorFlow version: {tf.__version__}")

# Check GPU availability
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
    print(f"GPU devices: {tf.config.list_physical_devices('GPU')}")
else:
    print("GPU not available, using CPU")

: 

In [ ]:
    # Setup paths and configuration
import os

    # Define project paths
PROJECT_ROOT = os.path.abspath(os.getcwd())
folder_name = os.path.join(PROJECT_ROOT, 'archive')
folder_name = os.path.normpath(folder_name)

# Path utility functions
def resolve_sequence_dir(base_path, sequence_name):
    """Find correct sequence directory path"""
    candidates = [
        os.path.join(base_path, 'train', sequence_name),
        os.path.join(base_path, 'train', 'train', sequence_name),
        os.path.join(base_path, 'val', sequence_name),
        os.path.join(base_path, 'val', 'val', sequence_name)
    ]
    
    for path in candidates:
        if os.path.exists(path) and os.path.isdir(path):
            return path
    return candidates[0]

def list_sequence_frames(sequence_dir):
    """Get sorted list of image frames in sequence directory"""
    if not os.path.exists(sequence_dir):
        return []
    return sorted([f for f in os.listdir(sequence_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

# Load CSV files
train_csv = os.path.join(folder_name, 'train.csv')
val_csv = os.path.join(folder_name, 'val.csv')

train_doc = np.random.permutation(open(train_csv).readlines())
val_doc = np.random.permutation(open(val_csv).readlines())

# Configuration
batch_size = 16  # Smaller batch size for better training
x, y, z = 100, 100, 30  # Image dimensions and sequence length
num_epochs = 20

print(f"Dataset path: {folder_name}")
print(f"Training samples: {len(train_doc)}")
print(f"Validation samples: {len(val_doc)}")

In [ ]:
# Data generator with MobileNet preprocessing
def generator_mobilenet_gru(source_path, folder_list, batch_size, x, y, z, normalise, augment=False):
    """Generator for MobileNet + GRU model"""
    print(f'Source path = {source_path}')
    
    while True:
        t = np.random.permutation(folder_list)
        
        for i in range(0, len(t), batch_size):
            batch_features = []
            batch_labels = []
            
            for folder in t[i:i + batch_size]:
                try:
                    # Extract sequence name and label
                    parts = folder.strip().split(';')
                    sequence_name = parts[0]
                    label = int(parts[2])
                    
                    # Get sequence directory
                    sequence_dir = resolve_sequence_dir(source_path, sequence_name)
                    imgs = list_sequence_frames(sequence_dir)
                    
                    if len(imgs) < z:
                        continue
                    
                    # Sample frames uniformly
                    source_frames = np.linspace(0, len(imgs) - 1, z).astype(int)
                    
                    batch_data = []
                    for frame_idx in source_frames:
                        image_path = os.path.join(sequence_dir, imgs[frame_idx])
                        
                        # Load and preprocess image
                        image = imageio.imread(image_path)
                        
                        # Resize for MobileNet (224x224)
                        image_resized = cv2.resize(image, (224, 224))
                        
                        # Normalize for MobileNet
                        if normalise:
                            image_normalized = tf.keras.applications.mobilenet_v2.preprocess_input(image_resized)
                        else:
                            image_normalized = image_resized / 255.0
                            
                        batch_data.append(image_normalized)
                    
                    batch_features.append(np.array(batch_data))
                    
                    # One-hot encode label (5 classes)
                    one_hot = np.zeros(5)
                    one_hot[label] = 1
                    batch_labels.append(one_hot)
                    
                except Exception as e:
                    print(f"Error processing {folder}: {e}")
                    continue
            
            if len(batch_features) > 0:
                yield np.array(batch_features), np.array(batch_labels)

print("Data generator defined successfully")

In [ ]:
# Model 8: Transfer Learning with MobileNet and GRU (Best performing model)
def create_mobilenet_gru_model(sequence_length=30):
    """Create MobileNet + GRU model for gesture recognition"""
    
    # Load pre-trained MobileNetV2
    base_model = MobileNetV2(input_shape=(224, 224, 3), 
                           include_top=False, 
                           weights='imagenet')
    base_model.trainable = False  # Freeze base model
    
    # Create the full model
    model = Sequential([
        TimeDistributed(base_model, input_shape=(sequence_length, 224, 224, 3)),
        TimeDistributed(GlobalAveragePooling2D()),
        TimeDistributed(Dense(128, activation='relu')),
        TimeDistributed(Dropout(0.5)),
        
        # GRU layers
        GRU(64, return_sequences=True, dropout=0.5),
        GRU(32, dropout=0.5),
        
        # Final classification layers
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(5, activation='softmax')  # 5 gesture classes
    ])
    
    return model

# Create model
model = create_mobilenet_gru_model(sequence_length=z)

# Compile model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, 
              loss='categorical_crossentropy', 
              metrics=['categorical_accuracy'])

# Display model summary
model.summary()

In [ ]:
# Setup callbacks (Fixed version)
curr_dt_time = datetime.datetime.now()
model_name = f'gesture_model_{curr_dt_time.strftime("%Y%m%d_%H%M%S")}'

if not os.path.exists(model_name):
    os.makedirs(model_name)

# Fixed callbacks - use .h5 format instead of .keras
checkpoint = ModelCheckpoint(
    filepath=os.path.join(model_name, 'best_model.h5'),  # Changed to .h5
    monitor='val_categorical_accuracy',
    save_best_only=True,
    save_weights_only=False,
    verbose=1,
    mode='max'  # Added mode parameter
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=0.00001,
    verbose=1
)

early_stop = EarlyStopping(
    monitor='val_categorical_accuracy',
    patience=10,
    restore_best_weights=True,
    verbose=1,
    mode='max'  # Added mode parameter
)

callbacks = [checkpoint, reduce_lr, early_stop]
print(f"Model will be saved to: {model_name}")

In [ ]:
# Create data generators
train_generator = generator_mobilenet_gru(
    folder_name, train_doc, batch_size, x, y, z, normalise=True, augment=True
)

val_generator = generator_mobilenet_gru(
    folder_name, val_doc, batch_size, x, y, z, normalise=True, augment=False
)

# Calculate steps
steps_per_epoch = len(train_doc) // batch_size
validation_steps = len(val_doc) // batch_size

print(f"Steps per epoch: {steps_per_epoch}")
print(f"Validation steps: {validation_steps}")

In [ ]:
# Train the model
print("Starting training...")
print(f"Training for {num_epochs} epochs with batch size {batch_size}")

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=num_epochs,
    validation_data=val_generator,
    validation_steps=validation_steps,
    callbacks=callbacks,
    verbose=1
)

print("Training completed!")

In [ ]:
# Plot training history
def plot_training_history(history):
    """Plot training and validation metrics"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot accuracy
    ax1.plot(history.history['categorical_accuracy'], label='Training Accuracy', color='blue')
    ax1.plot(history.history['val_categorical_accuracy'], label='Validation Accuracy', color='red')
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True)
    
    # Plot loss
    ax2.plot(history.history['loss'], label='Training Loss', color='blue')
    ax2.plot(history.history['val_loss'], label='Validation Loss', color='red')
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Print final results
    final_train_acc = max(history.history['categorical_accuracy'])
    final_val_acc = max(history.history['val_categorical_accuracy'])
    
    print(f"\n=== TRAINING RESULTS ===")
    print(f"Best Training Accuracy: {final_train_acc:.4f} ({final_train_acc*100:.2f}%)")
    print(f"Best Validation Accuracy: {final_val_acc:.4f} ({final_val_acc*100:.2f}%)")
    print(f"Model saved to: {model_name}")

# Plot results
plot_training_history(history)

In [ ]:
# Save final model (Fixed version)
model.save(os.path.join(model_name, 'final_gesture_model.h5'))  # Changed to .h5
print(f"Final model saved to: {os.path.join(model_name, 'final_gesture_model.h5')}")

# Save training history
import pickle
with open(os.path.join(model_name, 'training_history.pkl'), 'wb') as f:
    pickle.dump(history.history, f)
    
print("Training complete! Your gesture recognition model is ready.")
print("\n5 gesture classes supported:")
print("0: Thumbs Up")
print("1: Thumbs Down") 
print("2: Left Swipe")
print("3: Right Swipe")
print("4: Stop Gesture")